In [2]:
import pandas as pd

actual_df = pd.read_csv('./evaluation_test/test_labels.csv.zip')
predictions_df = pd.read_csv('./evaluation_test/tox_comment_predictions.csv')

In [3]:
actual_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [4]:
predictions_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.995929,0.527431,0.978008,0.103378,0.954302,0.493032
1,0000247867823ef7,0.001001,0.000225,0.000289,0.000187,0.000310,0.000231
2,00013b17ad220c46,0.000768,0.000258,0.000324,0.000191,0.000312,0.000270
3,00017563c3f7919a,0.000656,0.000307,0.000280,0.000263,0.000328,0.000266
4,00017695ad8997eb,0.001372,0.000194,0.000285,0.000175,0.000321,0.000214


In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
import string
import torchmetrics

from transformers import pipeline
from tqdm.notebook import tqdm
tqdm.pandas()

In [6]:
%pip install -q torchmetrics watermark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%load_ext watermark

In [8]:
%watermark --iversions

torchmetrics: 0.11.4
torch       : 2.0.0
pandas      : 1.4.3
transformers: 4.25.1
numpy       : 1.23.5
sklearn     : 1.1.2



In [9]:
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [10]:
actual_df_final = actual_df[(actual_df['toxic'] != -1) & (actual_df['severe_toxic'] != -1) & (actual_df['obscene'] != -1) & (actual_df['threat'] != -1) & (actual_df['insult'] != -1) & (actual_df['identity_hate'] != -1)]
actual_df_final

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0
...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0,0,0,0,0,0
153151,fff9d70fe0722906,0,0,0,0,0,0
153154,fffa8a11c4378854,0,0,0,0,0,0
153155,fffac2a094c8e0e2,1,0,1,0,1,0


In [11]:
predictions_df_final = predictions_df[predictions_df['id'].isin(actual_df_final['id'])]
predictions_df_final

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0.000852,0.000330,0.000264,0.000332,0.000411,0.000300
7,000247e83dcc1211,0.257406,0.000945,0.002013,0.004331,0.015416,0.002786
11,0002f87b16116a7f,0.420584,0.000954,0.057854,0.000486,0.050901,0.002374
13,0003e1cccfd5a40a,0.000500,0.000348,0.000336,0.000277,0.000345,0.000371
14,00059ace3e3e9a53,0.000575,0.000315,0.000305,0.000258,0.000316,0.000305
...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0.000838,0.000225,0.000274,0.000176,0.000340,0.000243
153151,fff9d70fe0722906,0.911454,0.016917,0.706170,0.002072,0.543538,0.006530
153154,fffa8a11c4378854,0.920124,0.028685,0.193242,0.011904,0.360335,0.600204
153155,fffac2a094c8e0e2,0.994180,0.292508,0.962748,0.025448,0.932789,0.437960


### AUC ROC

In [12]:
# Score for each cat

for cat in label_columns:
    
    print(f"Category: {cat}")
    print(f"Sklearn score: {metrics.roc_auc_score(actual_df_final[cat], predictions_df_final[cat])}")
    print(f"torchmetrics score: {torchmetrics.functional.classification.binary_auroc(torch.tensor(predictions_df_final[cat].values),torch.tensor(actual_df_final[cat].values), thresholds=None)}")
    print("#" * 30)
    print()

Category: toxic
Sklearn score: 0.97585272387152
torchmetrics score: 0.9758527278900146
##############################

Category: severe_toxic
Sklearn score: 0.9904595956768398
torchmetrics score: 0.9904595613479614
##############################

Category: obscene
Sklearn score: 0.9826870491427941
torchmetrics score: 0.9826870560646057
##############################

Category: threat
Sklearn score: 0.9948823608937069
torchmetrics score: 0.994882345199585
##############################

Category: insult
Sklearn score: 0.982746042462682
torchmetrics score: 0.9827460050582886
##############################

Category: identity_hate
Sklearn score: 0.9917913690261592
torchmetrics score: 0.9917913675308228
##############################



In [13]:
print(f"Sklearn score: {metrics.roc_auc_score(actual_df_final[label_columns].values, predictions_df_final[label_columns].values)}")
print(F"torchmetrics score: {torchmetrics.functional.classification.multilabel_auroc(torch.tensor(predictions_df_final[label_columns].values),torch.tensor(actual_df_final[label_columns].values),num_labels=6,thresholds=None)}")

print()
print(f"Sklearn score (weighted): {metrics.roc_auc_score(actual_df_final[label_columns].values, predictions_df_final[label_columns].values, average='weighted')}")
print(F"torchmetrics score (weighted): {torchmetrics.functional.classification.multilabel_auroc(torch.tensor(predictions_df_final[label_columns].values),torch.tensor(actual_df_final[label_columns].values),num_labels=6,thresholds=None,average='weighted' )}")

Sklearn score: 0.9864031901789504
torchmetrics score: 0.9864031672477722

Sklearn score (weighted): 0.9806515366803571
torchmetrics score (weighted): 0.9806516170501709


### Average Precision

In [15]:
# Score for each cat

for cat in label_columns:
    
    print(f"Category: {cat}")
    print(f"Sklearn score: {metrics.average_precision_score(actual_df_final[cat], predictions_df_final[cat])}")
    print(f"torchmetrics score: {torchmetrics.functional.classification.binary_average_precision(torch.tensor(predictions_df_final[cat].values),torch.tensor(actual_df_final[cat].values), thresholds=None)}")
    print("#" * 30)
    print()

Category: toxic
Sklearn score: 0.8216459600305824
torchmetrics score: 0.821645975112915
##############################

Category: severe_toxic
Sklearn score: 0.3667870282521481
torchmetrics score: 0.36678704619407654
##############################

Category: obscene
Sklearn score: 0.8091451961689659
torchmetrics score: 0.8091452121734619
##############################

Category: threat
Sklearn score: 0.5502490070911502
torchmetrics score: 0.5502489805221558
##############################

Category: insult
Sklearn score: 0.7981398062846693
torchmetrics score: 0.798139750957489
##############################

Category: identity_hate
Sklearn score: 0.6929935336916705
torchmetrics score: 0.6929935216903687
##############################



In [16]:
print(f"Sklearn score: {metrics.average_precision_score(actual_df_final[label_columns].values, predictions_df_final[label_columns].values)}")
print(F"torchmetrics score: {torchmetrics.functional.classification.multilabel_average_precision(torch.tensor(predictions_df_final[label_columns].values),torch.tensor(actual_df_final[label_columns].values),num_labels=6,thresholds=None)}")

print()
print(f"Sklearn score (weighted): {metrics.average_precision_score(actual_df_final[label_columns].values, predictions_df_final[label_columns].values, average='weighted')}")
print(F"torchmetrics score (weighted): {torchmetrics.functional.classification.multilabel_average_precision(torch.tensor(predictions_df_final[label_columns].values),torch.tensor(actual_df_final[label_columns].values),num_labels=6,thresholds=None,average='weighted' )}")

Sklearn score: 0.6731600885865311
torchmetrics score: 0.6731600761413574

Sklearn score (weighted): 0.7911248936154436
torchmetrics score (weighted): 0.7911249399185181
